In [1]:
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt
from scipy.stats import mode
import pandas as pd
import numpy as np

from sklearn.datasets import load_iris
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
from torch.utils.data import Dataset, DataLoader

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
iris = load_iris()

In [3]:
feature_names_full = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
feature_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']

In [4]:
df = pd.DataFrame(iris.data, columns = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width'])
df['class'] = iris.target

In [5]:
df

,sepal_length,sepal_width,petal_length,petal_width,class
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [6]:
df_max = df[feature_names].max(axis = 0)
df_min = df[feature_names].min(axis = 0)

df[feature_names] = (df[feature_names] - (df_max + df_min) / 2) # 데이터 중심 보정
df[feature_names] = df[feature_names] / df[feature_names].max(axis = 0) # 데이터 정규화

In [7]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size = 0.3, shuffle = True)

In [8]:
from sklearn.neighbors import KNeighborsClassifier

knn_classifier = KNeighborsClassifier(n_neighbors = 5)
knn_classifier.fit(df_train[feature_names], df_train['class'])

prediction = knn_classifier.predict(df_test[feature_names])

knn_pred = prediction

score = (prediction == df_test['class'].values).sum() / len(df_test)
score * 100

95.55555555555556

In [9]:
knn_classifier = KNeighborsClassifier(n_neighbors = 5)
knn_classifier.fit(df_train[feature_names], df_train['class'])

KNeighborsClassifier()

In [10]:
from sklearn.tree import DecisionTreeClassifier

decision_tree_classifier = DecisionTreeClassifier()
decision_tree_classifier.fit(df_train[feature_names_full], df_train['class'])

prediction = decision_tree_classifier.predict(df_test[feature_names_full])

dc_pred = prediction

score = (prediction == df_test['class'].values).sum() / len(df_test)
score * 100

91.11111111111111

In [11]:
from sklearn.linear_model import LogisticRegression

logistic_regression = LogisticRegression()
logistic_regression.fit(df_train[feature_names_full], df_train['class'])

prediction = logistic_regression.predict(df_test[feature_names_full])

lr_pred = prediction

score = (prediction == df_test['class'].values).sum() / len(df_test)
score * 100

95.55555555555556

In [12]:
data = df.to_numpy()
X, y = data[:, :4], data[:, -1]

y = np.eye(3)[y.astype(np.int32)]

In [13]:
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype = torch.float32)
        self.y = torch.tensor(y, dtype = torch.float32)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [14]:
class NN(nn.Module):
    def __init__(self, in_dim, out_dim):
        super(NN, self).__init__()
        
        self.fc1 = nn.Linear(in_dim, 16)
        self.batch_norm1 = nn.BatchNorm1d(16)
        self.relu1 = nn.ReLU()
        
        self.fc2 = nn.Linear(16, 16)
        self.batch_norm2 = nn.BatchNorm1d(16)
        self.relu2 = nn.ReLU()
        
        self.fc3 = nn.Linear(16, out_dim)
        self.softmax = nn.Softmax()
    
    def forward(self, x):
        x1 = self.fc1(x)
        x1 = self.batch_norm1(x1)
        x1 = self.relu1(x1)
        
        x2 = self.fc2(x1)
        x2 = self.batch_norm2(x2)
        x2 = self.relu2(x2)
        
        x3 = self.fc3(x2)
        x3 = self.softmax(x3)
        
        return x3

In [15]:
model = NN(4, 3).to(device)
model.load_state_dict(torch.load('./best_model/Epoch_2755_best_model.pth'))

<All keys matched successfully>

In [16]:
test_data = torch.tensor(df_test[feature_names_full].to_numpy(), dtype = torch.float32).to(device)
pred = model(test_data).cpu().detach().numpy()

C:\Users\user\AppData\Local\Temp\ipykernel_6072\1390529546.py:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x3 = self.softmax(x3)


In [17]:
(pred.argmax(1) == df_test['class']).sum() / len(df_test) * 100

97.77777777777777

In [18]:
nn_pred = pred

In [19]:
ensemble_pred = list()
for p1, p2, p3, p4 in zip(knn_pred, dc_pred, lr_pred, nn_pred.argmax(1)):
    ensemble_pred.append(mode([p1, p2, p3, p4]).mode[0])
ensemble_pred = np.array(ensemble_pred)

C:\Users\user\AppData\Local\Temp\ipykernel_6072\3655463648.py:3: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  ensemble_pred.append(mode([p1, p2, p3, p4]).mode[0])


In [20]:
ensemble_pred

array([2, 0, 0, 0, 2, 0, 2, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 2, 2, 2,
       0, 2, 1, 2, 2, 0, 0, 2, 2, 2, 2, 1, 2, 0, 0, 2, 1, 0, 1, 1, 1, 0,
       1], dtype=int64)

In [21]:
print((ensemble_pred == df_test['class'].to_numpy()).sum() / len(df_test) * 100)

95.55555555555556
